# Cube Packcage
The custom cube package is preinstalled.  just load the library and start to use it 
<hr/>


### Load Library and Login to CubeAPI

In [1]:
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

Loading required package: usethis

Loading required package: reshape


Attaching package: ‘rapportools’


The following objects are masked from ‘package:stats’:

    IQR, median, sd, var


The following objects are masked from ‘package:base’:

    max, mean, min, range, sum



Attaching package: ‘data.table’


The following object is masked from ‘package:reshape’:

    melt


Loading required package: tfruns



In [2]:
# create a CubeAPI object
cube_api = CubeAPI$new()

INFO [2021-01-06 13:14:02] Version: 0.0.2
INFO [2021-01-06 13:14:02] http://10.105.16.22/metadata-service/


In [ ]:
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

### Get Metadata Definition

In [3]:
# retrieve metadata element
response = cube_api$get_element()

INFO [2021-01-06 13:14:05] GET: http://10.105.16.22/metadata-service/metadata_definition/element/
INFO [2021-01-06 13:14:06] status_code: 200


In [ ]:
str(response)

In [4]:
# get the content of the response
content = content(response)

In [ ]:
str(content)

In [5]:
# to data frame
data.frame(do.call(rbind, content$results))[c("id", "element_label", "element_collection_label", "element_abbreviation")]

id,element_label,element_collection_label,element_abbreviation
<list>,<list>,<list>,<list>
129,About The Cube Initiative Page,About Page,00
85,Assay Type,Assay Types,AS
125,Biobank,Biobank,BB
137,Data Set,Data Sets,DS
115,Disease,Diseases,DS
112,Drug Treatment,Drug Treatments,DR
122,File Set,File Sets,FS
118,Funding Source,Funding Sources,FD
117,Human Donors,Human Donors,HU


### Get Bucket data from metadata collection

In [6]:
storage_info = cube_api$get_metadata_collection_storage_info()

INFO [2021-01-06 13:14:17] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-06 13:14:18] status_code: 200
INFO [2021-01-06 13:14:18] number of accession_ids: 2
INFO [2021-01-06 13:14:18] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-06 13:14:19] status_code: 200
INFO [2021-01-06 13:14:19] number of row returned: 1


In [7]:
storage_info

accession_id,uri,bucket_name,file_name
<chr>,<chr>,<chr>,<chr>
JAXDS0000A,gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_Bears_CBA_NMR_200723.txt,gs://jax-cube-prd-ctrl-01-project-data/study_3-1,Three_Bears_CBA_NMR_200723.txt


In [9]:
## get bucket data
bucket_data = get_bucket_data(bucket_name = storage_info[1, 3], file_name = storage_info[1, 4] )
bucket_data

Synchronizing gs://jax-cube-prd-ctrl-01-project-data/study_3-1 to local directory gs/jax-cube-prd-ctrl-01-project-data/study_3-1




both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Building synchronization state...
Starting synchronization...
Copying gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_Bears_Water_Intake_9.15.2020.txt...
Copying gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_Bears_CBA_GTT_200723.txt...
Copying gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_Bears_CBA_NMR_200723.txt...
Copying gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_Bears_Food_Intake_9.3.2020.txt...
Copying gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_Bears_Body_Weight_9.3.2020.txt...
Copying gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_bears_CBA_BODY_WEIGHT_200723.txt...
/ [6/6 files][965.6 KiB/965.6 KiB] 100% Done            

X,Barcode,JAX_ASSAY_PENBARCODE,JAX_ASSAY_BLINDID,JAX_ASSAY_PRIMARYID,JAX_ASSAY_PRIMARYIDVALUE,JAX_ASSAY_SECONDAYID,JAX_ASSAY_SECONDARYIDVALUE,JAX_ASSAY_SEX,JAX_ASSAY_COATCOLOR,⋯,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.JAX_EMPLOYEE_ACTIVEWEBONLYACCOUNT,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.CI_USERNAME,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.STATUS,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.ENFORCE_PROJECT_SECURITY,CLIMB_ID,sex,line,strain,diet,treatment
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,⋯,<lgl>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
0,XCP3452,PN8712,3B0101,Ear Notch,R,NA,NA,F,black (non-agouti),⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3693,Female,JR000664,C57BL/6J,10% fat + fiber,None
1,XCP3453,PN8713,3B0112,Ear Notch,R,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3773,Female,JR002105,NZO/HILtJ,10% fat + fiber,None
2,XCP3454,PN8714,3B0103,Ear Notch,R,NA,NA,M,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3758,Male,JR000928,CAST/EiJ,44% fat+ fiber,None
3,XCP3455,PN8715,3B0104,Ear Notch,2B,NA,NA,M,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3796,Male,JR002105,NZO/HILtJ,44% fat+ fiber,None
4,XCP3456,PN8716,3B0105,Ear Notch,R,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3738,Female,JR000928,CAST/EiJ,44% fat+ fiber,None
5,XCP3457,PN8717,3B0106,Ear Notch,L,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3774,Female,JR002105,NZO/HILtJ,10% fat + fiber,None
6,XCP3458,PN8718,3B0107,Ear Notch,R,NA,NA,M,black (non-agouti),⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3713,Male,JR000664,C57BL/6J,10% fat + fiber,None
7,XCP3459,PN8719,3B0108,Ear Notch,2L,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3778,Female,JR002105,NZO/HILtJ,44% fat+ fiber,None
8,XCP3460,PN8720,3B0109,Ear Notch,R,NA,NA,M,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3790,Male,JR002105,NZO/HILtJ,10% fat + fiber,None


<br/><br/>

### Get Metadata Definition with a Parameter

In [ ]:
response = cube_api$get_element(element_id = 85)

In [ ]:
str(content(response))

### Get Metadata with accession ids 

In [ ]:
response = cube_api$get_element_instance(element_id = 85, accession_ids = c("JAXAS00031", "JAXAS00032", "JAXAS00034"))

In [ ]:
str(content(response)$results)

### Get Metadata Collection

In [ ]:
response = cube_api$get_metadata_collection()

In [ ]:
json = response_json_to_data(response)

In [ ]:
str(json$results)

In [ ]:
accession_ids = json$results[,2][[1]]$accession_id
accession_ids

In [ ]:
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

In [ ]:
results = content(response)$results
results[[4]][[4]]

In [ ]:
data_url = results[[4]][[4]][[1]][[4]]
url_parts = str_split(data_url, "/")
bucket_name = paste0("gs://", url_parts[[1]][4], "/", url_parts[[1]][5])
file_name =  url_parts[[1]][6]

# load the data from the GCP bucket
data_dir = gs_data_dir( bucket_name )

# read the file into a dataframe
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
df

### Call Any Get endpoint
Call any get end point.  All the endpoints are listed here: https://metadata-service-sqa.azurewebsites.net/swagger/

In [ ]:
# specify a end point
response = cube_api$get_end_point("metadata_definition/property/?element_id=85")

In [ ]:
content(response)$count

In [ ]:
# another end point
response = cube_api$get_end_point("/metadata_repository/collection_item/1/")

In [ ]:
content(response)

### Post endpoint

In [ ]:
response = cube_api$post_element_instance_filter(element_id = 122, page = 1, page_size = 2)

In [ ]:
str(content(response))

### More Information

In [ ]:
?CubeAPI

In [ ]:
?Cube